In [89]:
#############
# LIBRARIES #
#############

from get_strava_data import my_data, process_data, bike_data, get_elevation # Functions to retrive data using strava api and process for visualizations

import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [90]:
# Opening activities data

processed_data = pd.read_csv('./data/processed_data.csv')
processed_data.head()

,name,distance,moving_time,elapsed_time,total_elevation_gain,type,id,start_date_local,achievement_count,kudos_count,...,average_watts,max_watts,weighted_average_watts,kilojoules,elev_high,year,month,day,weekday,hour
0,HIIT,0.0,0.5,0.5,0.0,Workout,6967778843,2022-04-11 21:40:51+00:00,0,5,...,NaN,NaN,NaN,NaN,NaN,2022,4,11,Monday,21
1,Afternoon Ride,18.1,1.1,1.2,167.3,Ride,6962213592,2022-04-10 12:52:32+00:00,24,12,...,143.0,522.0,153.0,555.6,71.5,2022,4,10,Sunday,12
2,Fun with the kiddos,4.0,0.6,2.2,7.5,Ride,6956840241,2022-04-09 14:47:48+00:00,1,7,...,37.2,NaN,NaN,78.3,14.8,2022,4,9,Saturday,14
3,Monterey - Butters,20.4,1.5,1.6,1355.0,Ride,6955866591,2022-04-09 09:55:44+00:00,28,15,...,140.9,1007.0,166.0,746.4,1154.2,2022,4,9,Saturday,9
4,Evening Activity,0.0,0.9,0.9,0.0,WeightTraining,6951585517,2022-04-08 18:37:40+00:00,0,1,...,NaN,NaN,NaN,NaN,NaN,2022,4,8,Friday,18


In [91]:
processed_data = processed_data[processed_data.type =='Ride']

grouped_by_day_hour = processed_data.groupby(['weekday', 'hour']).agg({'id': 'count'}).reset_index()

# Custom sorting weekdays starting on monday
day_order = CategoricalDtype(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
grouped_by_day_hour['weekday'] = grouped_by_day_hour['weekday'].astype(day_order)
grouped_by_day_hour.sort_values('weekday', inplace=True)

# Creating new entries for hours with no data
mux = pd.MultiIndex.from_product([grouped_by_day_hour.weekday.unique(), range(0,24)], names=['weekday', 'hour'])
grouped_by_day_hour = grouped_by_day_hour.set_index(['weekday', 'hour']).reindex(mux, fill_value=0).reset_index()

# Pivoting table to get the number of activities per hour
pivot_data = grouped_by_day_hour.pivot_table(index='weekday', columns='hour', values='id', aggfunc='sum')
pivot_data.fillna(0, inplace=True)
pivot_data


hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
weekday,,,,,,,,,,,,,,,,,,,,,
Monday,0,0,0,0,0,0,0,0,1,0,...,1,1,5,5,17,3,3,0,0,0
Tuesday,0,0,0,0,0,0,0,0,0,0,...,2,5,5,11,10,6,0,0,0,0
Wednesday,0,0,0,0,0,0,0,0,0,0,...,2,0,2,16,12,6,0,1,2,0
Thursday,0,0,0,0,0,0,0,0,0,0,...,1,4,5,18,19,4,1,7,0,0
Friday,0,1,0,0,0,1,0,0,0,1,...,9,8,5,8,7,1,1,0,0,0
Saturday,0,0,0,0,1,1,2,5,12,14,...,20,15,11,6,6,1,3,1,0,0
Sunday,0,0,0,0,0,1,5,3,8,16,...,16,8,9,9,3,2,2,2,0,0


In [92]:
# Creating heatmap of workouts by weekday and hour
hours_of_day = ['12a', '1am', '2am', '3am', '4am', '5am', '6am', '7am', '8am', '9am', '10am', '11am', '12pm', '1pm', '2pm', '3pm', '4pm', '5pm', '6pm', '7pm', '8pm', '9pm', '10p', '11p']

fig = px.imshow(pivot_data,
labels=dict(x="", y=""),
color_continuous_scale='oranges',
x=hours_of_day,
aspect="auto"
)
fig.update_xaxes(side="top")
fig.update_layout(
    title='',
    xaxis_nticks=8,
    showlegend = False,
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [93]:
processed_data = processed_data[processed_data.type =='Ride']

grouped_by_month_day = processed_data.groupby(['month', 'day']).agg({'id': 'count'}).reset_index()
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
# Custom sorting weekdays starting on monday
month_order = CategoricalDtype(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], ordered=True)
# Converting month number to month name
grouped_by_month_day['month'] = grouped_by_month_day['month'].apply(lambda x: months[x-1])
grouped_by_month_day['month'] = grouped_by_month_day['month'].astype(month_order)
grouped_by_month_day.sort_values('month', inplace=True)

# # Creating new entries for hours with no data
# mux = pd.MultiIndex.from_product([grouped_by_month_day.month.unique(), range(0,24)], names=['month', 'day'])
# grouped_by_month_day = grouped_by_month_day.set_index(['month', 'day']).reindex(mux, fill_value=0).reset_index()

# Pivoting table to get the number of activities per hour
pivot_data_month_day = grouped_by_month_day.pivot_table(index='month', columns='day', values='id', aggfunc='sum')
pivot_data_month_day.fillna(0, inplace=True)
pivot_data_month_day

day,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
month,,,,,,,,,,,,,,,,,,,,,
January,0,0,0,1,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,1
February,0,0,0,0,1,1,0,0,0,0,...,0,1,0,0,0,2,0,0,0,0
March,1,1,0,0,1,1,0,0,0,0,...,2,1,0,0,3,2,1,1,2,2
April,0,3,3,1,1,2,2,1,3,2,...,1,2,3,2,3,2,1,1,0,0
May,3,2,3,2,2,2,4,2,2,2,...,2,2,4,3,5,2,1,2,3,2
June,5,3,4,4,1,3,4,0,2,4,...,2,3,4,2,1,3,1,1,1,0
July,2,2,2,3,3,2,0,6,2,3,...,4,1,4,3,5,4,3,2,4,3
August,4,7,0,3,2,3,2,2,4,1,...,2,2,0,1,3,4,2,4,4,1
September,2,2,2,2,2,3,0,2,4,1,...,1,1,2,1,3,2,1,0,4,0


In [94]:
# Creating heatmap of workouts by month and day

fig = px.imshow(pivot_data_month_day,
labels=dict(x="", y="", color="Activities"),
color_continuous_scale='oranges',
# x=hours_of_day,
aspect="auto"
)
fig.update_xaxes(side="top")
fig.update_layout(
    title='',
    xaxis_nticks=8,
    showlegend = False,
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [95]:
processed_data.columns

Index(['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'id', 'start_date_local',
       'achievement_count', 'kudos_count', 'comment_count', 'photo_count',
       'map', 'trainer', 'gear_id', 'average_speed', 'max_speed',
       'average_heartrate', 'max_heartrate', 'pr_count', 'total_photo_count',
       'suffer_score', 'average_cadence', 'average_temp', 'average_watts',
       'max_watts', 'weighted_average_watts', 'kilojoules', 'elev_high',
       'year', 'month', 'day', 'weekday', 'hour'],
      dtype='object')

In [88]:
######################
# PERFORMACE METRICS #
######################

# Scatterplots
px.scatter(processed_data, x='year', y='total_elevation_gain', size='distance', color='year', title='Average Speed vs Average Cadence', trendline="ols")

In [ ]:
#